In [1]:
import geemap, ee

ee.Initialize()

roi = geemap.shp_to_ee(r'D:/work/new/shp/fw.shp')
fw = roi.geometry()

sample = geemap.shp_to_ee(r'D:/work/new/point/result.shp')
Map = geemap.Map()
# Map

In [2]:
def fun(image):
    return (image.select('tmmx').add(image.select('tmmn'))).multiply(0.5).rename('tem')

def NDVI(img):
    ndvi = img.normalizedDifference(['SR_B4', 'SR_B3']).rename('NDVI')
    return img.addBands(ndvi)

def EVI(img):
    nir = img.select("SR_B4")
    red = img.select("SR_B3")
    blue = img.select("SR_B1")
    evi = img.expression("2.5*(B4 - B3)/(B4 + 6*B3 - 7.5*B1 + 1)",{"B4": nir,"B3": red ,"B1": blue})
    evi = evi.rename('EVI')
    return img.addBands(evi)

def EVI2(img):
    nir = img.select("SR_B4")
    red = img.select("SR_B3")
    evi2 = img.expression("2.5*(B4 - B3)/(B4 + 2.4*B3 + 1)",{"B4": nir,"B3": red})
    evi2 = evi2.rename('EVI2')
    return img.addBands(evi2)

def LSWI(img):
    lswi = img.normalizedDifference(['SR_B4', 'SR_B5']).rename('LSWI')
    return img.addBands(lswi)

def GCVI(img):
    nir = img.select("SR_B4")
    green = img.select("SR_B2")
    gcvi = img.expression("B4/B2-1",{"B4": nir,"B2": green})
    gcvi = gcvi.rename('GCVI')
    return img.addBands(gcvi)

def RVI(img):
    nir = img.select("SR_B4")
    red = img.select("SR_B3")
    rvi = img.expression("B4/B3",{"B4": nir,"B3": red})
    rvi = rvi.rename('RVI')
    return img.addBands(rvi)

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0);

#   // Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0);
    
    return image.addBands(opticalBands, None, True)\
                 .addBands(thermalBands, None, True)\
                 .updateMask(qaMask)\
                 .updateMask(saturationMask)\
                 .copyProperties(image, ["system:time_start",'system:id'])

def K(image):
    return image.expression(
        'pr/aet',{
            'pr':image.select('pr'),
            'aet':image.select('aet').multiply(0.1)
        }
    ).rename('K')

# 投影
def project(image):
    return image.reproject(ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE").first().select(0).projection())

In [3]:
year  = 2020
images = []
bands = ['EVI','pr','tem','elevation','slope','NDVI']
label = 'CID'

# CRU  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
            .filterBounds(fw) \
            .filter(ee.Filter.calendarRange(year,year,'year')) \
            .filterMetadata('CLOUD_COVER', 'less_than', 25)\
            .map(maskL8sr) \
            .map(NDVI).map(EVI).map(RVI).mosaic()
images.append(collection.select('NDVI'))
images.append(collection.select('EVI'))
images.append(collection.select('RVI'))

temperature = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterBounds(fw)\
                .filter(ee.Filter.calendarRange(year,year,'year'))\
                .select(['tmmn','tmmx'])\
                .map(fun).mean().multiply(0.1)\
                .clip(fw)
images.append(project(temperature))

precipitation = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
                .filterBounds(fw)\
                .filter(ee.Filter.calendarRange(year,year,'year'))\
                .select('pr')\
                .sum().clip(fw)
images.append(project(precipitation))

Kdata = ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")\
            .filterBounds(fw)\
            .filter(ee.Filter.calendarRange(year, year, 'year'))\
            .select(['pr','aet'])
Ks = Kdata.map(K).mean().float().clip(fw)
images.append(project(Ks))

dataset = ee.Image('USGS/SRTMGL1_003').clip(fw)
elevation = dataset.select('elevation')
slope = ee.Terrain.slope(elevation).rename('slope')
images.append(elevation)
images.append(slope)

In [4]:
image = ee.Image(images)

In [5]:
# 特征值的提取
training = image.select(bands).sampleRegions(**{
    'collection':sample,
    'scale':30,
    'geometries': True
})

#  random uniforms to the training dataset.
withRandom = training.randomColumn() #样本点随机的排列

# 我们想保留一些数据进行测试，以避免模型过度拟合
split = 0.70
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))#筛选70%的样本作为训练样本
testingPartition = withRandom.filter(ee.Filter.gte('random', split))#筛选30%的样本作为测试样本

In [6]:
# 分类方法选择smileCart() randomForest() minimumDistance libsvm
classifier = ee.Classifier.smileRandomForest(120).train(trainingPartition,label,bands)
classified = image.select(bands).classify(classifier)

In [7]:
# 运用测试样本分类，确定要进行函数运算的数据集以及函数
test = testingPartition.classify(classifier)
test_accuracy = test.errorMatrix(label, 'classification')
print('总体精度:',test_accuracy.accuracy().getInfo())

总体精度: 0.5944584382871536
